In [19]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream, duration
from keras import backend as K
from keras import Input
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Permute
from keras.layers import RepeatVector
from keras.layers import Multiply
from keras.layers import Lambda
#from keras.optimizers import RMSprop
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
#tf.config.list_logical_devices('GPU')
#from tensorflow.python.client import device_lib 
#print(device_lib.list_local_devices())

In [48]:
def create_network(notes_in, durations_in, n_notes, n_durations):
    batch_size = 64
    embedding_dim = 100 #100
    rnn_units = 256 #256
    x1 = Embedding(n_notes, embedding_dim)(notes_in)
    x2 = Embedding(n_durations, embedding_dim)(durations_in)

    x = Concatenate()([x1,x2])

    x = LSTM(rnn_units, return_sequences=True)(x)
    x = LSTM(rnn_units, return_sequences=True)(x)
#     x = BatchNorm()(x) #new
#    x = Dropout(0.3)(x) #new
    #x = Dense(12)(x) #new
    #x = Activation('relu')(x) #new

    e = Dense(1, activation='relu')(x)#e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)

    alpha = Activation('softmax')(e) #'softmax'

    c = Permute([2,1])(RepeatVector(rnn_units)(alpha))
    c = Multiply()([x,c])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)

    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c) #softmax
    durations_out = Dense(n_durations, activation = 'softmax', name='duration')(c)

    print(notes_out)
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    att_model = Model([notes_in, durations_in], alpha)

    opti = RMSprop(learning_rate = 0.0001) #0.001 RMSProp
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti, metrics = ['accuracy'])
    
    return model, att_model


#create_network()

In [57]:
def get_notes():

    notes = []
    durations = []
    for file in glob.glob("midi/jazz2/*.mid"):
        try:
            midi = converter.parse(file)
            print("Parsing %s" % file)
            notes_to_parse = None
        #try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except Exception as inst:# file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            print(inst)
            pass
        
        #print(notes_to_parse)
        for element in notes_to_parse:
            #print(element)
            if isinstance(element, note.Note):
                if element.isRest:
                    notes.append(str(element.name))
                    durations.append(element.duration.quarterLength)
                else:
                    notes.append(str(element.pitch))
                    durations.append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                durations.append(element.duration.quarterLength)

    print("durations", durations)
    pickle.dump(notes, open('notes.p', 'wb'))
    pickle.dump(durations, open('durations.p', 'wb'))
    
    #print("notes: ", notes, "\n")
    return notes, durations


def prepare_sequences(notes, durations, n_vocab, d_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32 

    pitchnames = sorted(set(item for item in notes))
    #print("pitchnames: ", pitchnames, "\n")
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    #print("note_to_int: ", note_to_int, "\n")
    
    durationnames = sorted(set(item for item in durations))
    #print("pitchnames: ", pitchnames, "\n")
    duration_to_int = dict((note, number) for number, note in enumerate(durationnames))
    #print("note_to_int: ", note_to_int, "\n")

    network_input = []
    network_output = []
    network_input2 = []
    network_output2 = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        #print("sequence_in: ", sequence_in)
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        #print("network_input: ", network_input)
        network_output.append(note_to_int[sequence_out])
        
    for i in range(0, len(durations) - sequence_length, 1):
        sequence_in = durations[i:i + sequence_length]
        #print("sequence_in: ", sequence_in)
        sequence_out = durations[i + sequence_length]
        network_input2.append([duration_to_int[char] for char in sequence_in])
        #print("network_input: ", network_input)
        network_output2.append(duration_to_int[sequence_out])
        
    #print("network input", network_input)  
    #print("network input", network_input2)
    #print("network output", network_output)
    #print("network output", network_output2)

    n_patterns = len(network_input)
    n_patterns2 = len(network_input2)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input2 = np.reshape(network_input2, (n_patterns2, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_input2 = network_input2 / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    network_output2 = np_utils.to_categorical(network_output2)

    return (network_input, network_output, network_input2, network_output2)

In [58]:
notes, durations = get_notes()

n_vocab = len(set(notes))
d_vocab = len(set(durations))

Parsing midi/jazz2/OrnetteColeman_Chronology_FINAL.mid
Parsing midi/jazz2/StanGetz_BodyAndSoul_FINAL.mid
Parsing midi/jazz2/ColemanHawkins_Stompin'AtTheSavoy_FINAL.mid
Parsing midi/jazz2/KennyWheeler_DoubleVision_FINAL.mid
Parsing midi/jazz2/LeeKonitz_BopGoesToLeesel_FINAL.mid
Parsing midi/jazz2/FreddieHubbard_SpeakNoEvil_FINAL.mid
Parsing midi/jazz2/MilesDavis_Walkin'_FINAL.mid
Parsing midi/jazz2/ArtPepper_Stardust-2_FINAL.mid
Parsing midi/jazz2/LeeKonitz_Crosscurrent_FINAL.mid
Parsing midi/jazz2/VonFreeman_PassItOn_FINAL.mid
Parsing midi/jazz2/MilesDavis_SoWhat_FINAL.mid
Parsing midi/jazz2/MiltJackson_AllTheThingsYouAre_FINAL.mid
Parsing midi/jazz2/PaulDesmond_AloneTogether_FINAL.mid
Parsing midi/jazz2/PhilWoods_CrazyRhythm_FINAL.mid
Parsing midi/jazz2/BennyCarter_SweetLorraine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_BeginTheBeguine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_Milestones_FINAL.mid
Parsing midi/jazz2/GerryMulligan_LineForLyons_FINAL.mid
Parsing midi/jazz2/BobBerg_Natur

Parsing midi/jazz2/HerbieHancock_Orbits_FINAL.mid
Parsing midi/jazz2/JohnColtrane_Mr.P.C._FINAL.mid
Parsing midi/jazz2/SteveColeman_TakeTheColtrane_FINAL.mid
Parsing midi/jazz2/WoodyShaw_Imagination_FINAL.mid
Parsing midi/jazz2/JoeLovano_Work_FINAL.mid
Parsing midi/jazz2/KennyGarrett_BrotherHubbard-2_FINAL.mid
Parsing midi/jazz2/BuckClayton_DestinationK.C._FINAL.mid
Parsing midi/jazz2/JoeHenderson_In'NOut-1_FINAL.mid
Parsing midi/jazz2/HerbieHancock_Dolores_FINAL.mid
Parsing midi/jazz2/LeeKonitz_MeanToMe_FINAL.mid
Parsing midi/jazz2/DonEllis_YouSteppedOutOfADream-2_FINAL.mid
Parsing midi/jazz2/DonByas_BodyAndSoul_FINAL.mid
Parsing midi/jazz2/OrnetteColeman_Ramblin'_FINAL.mid
Parsing midi/jazz2/CharlieParker_OutOfNowhere_FINAL.mid
Parsing midi/jazz2/SidneyBechet_LimehouseBlues_FINAL.mid
Parsing midi/jazz2/FatsNavarro_OurDelight_FINAL.mid
Parsing midi/jazz2/SonnyRollins_VierdBlues_FINAL.mid
Parsing midi/jazz2/WarneMarsh_Wow-1_FINAL.mid
Parsing midi/jazz2/DonByas_CognacBlues_FINAL.mid
Par

Parsing midi/jazz2/BobBerg_YouAndTheNightAndTheMusic_FINAL.mid
Parsing midi/jazz2/DonByas_OutOfNowhere_FINAL.mid
Parsing midi/jazz2/LeeKonitz_Marshmallow_FINAL.mid
Parsing midi/jazz2/HarryEdison_DidYouCallHerToday_FINAL.mid
Parsing midi/jazz2/ColemanHawkins_It'sOnlyAPapermoon_FINAL.mid
Parsing midi/jazz2/JohnAbercrombie_Ralph'sPianoWaltz_FINAL.mid
Parsing midi/jazz2/RoyEldridge_St.LouisBlues_FINAL.mid
Parsing midi/jazz2/ChrisPotter_Togo_FINAL.mid
Parsing midi/jazz2/SteveColeman_DoubleVision_FINAL.mid
Parsing midi/jazz2/SonnyStitt_BlueMode_FINAL.mid
Parsing midi/jazz2/RedGarland_Oleo_FINAL.mid
Parsing midi/jazz2/SteveColeman_TheOracle-1_FINAL.mid
Parsing midi/jazz2/BennyGoodman_Runnin'Wild_FINAL.mid
Parsing midi/jazz2/WyntonMarsalis_Caravan_FINAL.mid
Parsing midi/jazz2/ChetBaker_ThereWillNeverBeAnotherYou-2_FINAL.mid
Parsing midi/jazz2/HankMobley_Remember_FINAL.mid
Parsing midi/jazz2/CliffordBrown_I'llRememberApril_AlternateTake2_FINAL.mid
Parsing midi/jazz2/KennyDorham_BluesInBe=Bop_FI

In [59]:
def train_network():
    """ Train a Neural Network to generate music """
    network_input, network_output, network_input2, network_output2 = prepare_sequences(notes, durations, n_vocab, d_vocab)
    
    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    #model = create_network(network_input2, d_vocab)
    model, att_model = create_network(notes_in, durations_in, n_vocab, d_vocab)

 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    ) 
    
    callbacks_list = [checkpoint]

    # Your line of code here
    #model.fit(network_input2, network_output2, epochs=5, callbacks=callbacks_list)
    model.fit([network_input, network_input2], [network_output, network_output2], batch_size=256, epochs=25, callbacks=callbacks_list)
    #model.summary()


train_network()
print("Finished Training")

Tensor("pitch/Softmax_16:0", shape=(None, 116), dtype=float32)
Epoch 1/25
769/769 [==============================] - 21s 28ms/step - loss: 5.1165 - pitch_loss: 3.4416 - duration_loss: 1.6749 - pitch_accuracy: 0.0728 - duration_accuracy: 0.4005
Epoch 2/25
769/769 [==============================] - 21s 28ms/step - loss: 5.0094 - pitch_loss: 3.3865 - duration_loss: 1.6229 - pitch_accuracy: 0.0747 - duration_accuracy: 0.4010
Epoch 3/25
769/769 [==============================] - 21s 28ms/step - loss: 5.0074 - pitch_loss: 3.3852 - duration_loss: 1.6222 - pitch_accuracy: 0.0752 - duration_accuracy: 0.4010
Epoch 4/25
769/769 [==============================] - 21s 28ms/step - loss: 5.0065 - pitch_loss: 3.3847 - duration_loss: 1.6218 - pitch_accuracy: 0.0756 - duration_accuracy: 0.4010
Epoch 5/25
769/769 [==============================] - 21s 28ms/step - loss: 5.0061 - pitch_loss: 3.3846 - duration_loss: 1.6216 - pitch_accuracy: 0.0755 - duration_accuracy: 0.4010
Epoch 6/25
769/769 [============

In [61]:
def prepare_sequences_prediction(notes, pitchnames, durations, durationnames, n_vocab, d_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    duration_to_int = dict((dur, number) for number, dur in enumerate(durationnames))
    sequence_length = 32
    network_input = []
    network_input2 = []
    output = []
    output2 = []
    
    print(duration_to_int)
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])
        
    for i in range(0, len(durations) - sequence_length, 1):
        sequence_in = durations[i:i + sequence_length]
        sequence_out = durations[i + sequence_length]
        network_input2.append([duration_to_int[char] for char in sequence_in])
        output2.append(duration_to_int[sequence_out])
 
    n_patterns = len(network_input)
    n_patterns2 = len(network_input2)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    normalized_input2 = np.reshape(network_input2, (n_patterns2, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    normalized_input2 = normalized_input2 / float(n_vocab)

    return (network_input, normalized_input, network_input2, normalized_input2)

def generate_notes(model, network_input, network_input2, pitchnames, durationnames, n_vocab, d_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    import random
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)
    start2 = np.random.randint(0, len(network_input2)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    int_to_duration = dict((number, dur) for number, dur in enumerate(durationnames))

    pattern = network_input[start]
    pattern2 = network_input2[start]
    prediction_output = []
    prediction_output2 = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input2 = np.reshape(pattern2, (1, len(pattern2), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction_input2 = prediction_input2 / float(d_vocab)
        
        ### Copy the line below from your above implementation.
        model.summary()
        prediction, prediction2 = model.predict([prediction_input, prediction_input2])
        print("prediction", prediction.shape)
        
        if(note_index %2 == 0):
            indices = list(np.nonzero(prediction[0])[0])
            indices2 = list(np.nonzero(prediction2[0])[0])
            #indices2 = list(np.nonzero(prediction2[0])[0])
            probs = prediction[0]
            probs2 = prediction2[0]
            index = random.choices(indices, weights=probs, k=1)
            index2 = random.choices(indices2, weights=probs2, k=1)
            #index2 = random.choice(indices2)
            index = index[0]
            index2 = index2[0]
        else:
            index = np.argmax(prediction)
            index2 = np.argmax(prediction2)

        result = int_to_note[index]
        result2 = int_to_duration[index2]
        prediction_output.append(result)
        prediction_output2.append(result2)
        
        pattern.append(index)
        pattern2.append(index2)
        pattern = pattern[1:len(pattern)]
        pattern2 = pattern2[1:len(pattern2)]
        print("pattern", pattern)
        print("pattern2", pattern2)

    return prediction_output, prediction_output2

In [62]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    durations = pickle.load(open('durations.p', 'rb'))
    
    pitchnames = sorted(set(item for item in notes))
    durationnames = sorted(set(item for item in durations))
    
    n_vocab = len(set(notes))
    d_vocab = len(set(durations))

    network_input, normalized_input, network_input2, normalized_input2 = prepare_sequences_prediction(notes, pitchnames, durations, durationnames, n_vocab, d_vocab)
    
    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    model, att_model = create_network(notes_in, durations_in, n_vocab, d_vocab)
    
    ### Add a line to load the weights here
    model.load_weights("weights2-improvement-14-5.0047-bigger.hdf5")
    
    

    prediction_output, prediction_output2 = generate_notes(model, network_input, network_input2, pitchnames, durationnames, n_vocab, d_vocab)
    create_midi(prediction_output, prediction_output2)

In [63]:
def create_midi(prediction_output, prediction_output2):
    offset = 0
    output_notes = []
    for pattern, pattern2 in zip(prediction_output, prediction_output2):
        off = pattern2
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Saxophone()
                notes.append(new_note)
            d = duration.Duration(pattern2)
            print(d)
            new_chord = chord.Chord(notes, duration=d)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == 'rest':
            new_note = note.Rest()
            d = duration.Duration(pattern2)
            new_note.duration = d
            new_note.storedInstrument = instrument.Saxophone()
            output_notes.append(new_note)
        else:
            d = duration.Duration(pattern2)
            print(d)
            new_note = note.Note(pattern, duration=d)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Saxophone()
            output_notes.append(new_note)
        offset += off
        
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()
print("done")

{0.0: 0, 0.25: 1, Fraction(1, 3): 2, 0.5: 3, Fraction(2, 3): 4, 0.75: 5, 1.0: 6, 1.25: 7, Fraction(4, 3): 8, 1.5: 9, Fraction(5, 3): 10, 1.75: 11, 2.0: 12, 2.25: 13, Fraction(7, 3): 14, 2.5: 15, Fraction(8, 3): 16, 2.75: 17, 3.0: 18, 3.25: 19, Fraction(10, 3): 20, 3.5: 21, Fraction(11, 3): 22, 3.75: 23, 4.0: 24, 4.25: 25, Fraction(13, 3): 26, 4.5: 27, Fraction(14, 3): 28, 4.75: 29, 5.0: 30, 5.25: 31, Fraction(16, 3): 32, 5.5: 33, Fraction(17, 3): 34, 5.75: 35, 6.0: 36, 6.25: 37, Fraction(19, 3): 38, 6.5: 39, Fraction(20, 3): 40, 6.75: 41, 7.0: 42, 7.25: 43, Fraction(22, 3): 44, 7.5: 45, Fraction(23, 3): 46, 7.75: 47, 8.0: 48, 8.25: 49, Fraction(25, 3): 50, 8.75: 51, 9.0: 52, 9.25: 53, Fraction(28, 3): 54, Fraction(31, 3): 55, 10.75: 56, 11.0: 57, Fraction(34, 3): 58, 12.25: 59, 12.5: 60, Fraction(38, 3): 61, 12.75: 62, 14.25: 63, Fraction(43, 3): 64, 15.5: 65, 16.25: 66, Fraction(50, 3): 67, 28.25: 68, 30.0: 69}
Tensor("pitch/Softmax_17:0", shape=(None, 116), dtype=float32)
Model: "fun

prediction (1, 116)
pattern [113, 86, 74, 108, 102, 80, 57, 97, 113, 108, 53, 58, 113, 113, 86, 57, 57, 62, 74, 108, 102, 97, 113, 86, 57, 57, 102, 86, 101, 113, 85, 113]
pattern2 [2, 2, 1, 4, 2, 1, 2, 2, 2, 2, 3, 3, 5, 3, 3, 5, 3, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 6, 3, 1, 1, 1]
Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   
_______

prediction (1, 116)
pattern [102, 80, 57, 97, 113, 108, 53, 58, 113, 113, 86, 57, 57, 62, 74, 108, 102, 97, 113, 86, 57, 57, 102, 86, 101, 113, 85, 113, 53, 113, 80, 113]
pattern2 [2, 1, 2, 2, 2, 2, 3, 3, 5, 3, 3, 5, 3, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 6, 3, 1, 1, 1, 2, 1, 1, 1]
Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   
_______

                                                                 permute_18[0][0]                 
__________________________________________________________________________________________________
lambda_17 (Lambda)              (None, 256)          0           multiply_18[0][0]                
__________________________________________________________________________________________________
pitch (Dense)                   (None, 116)          29812       lambda_17[0][0]                  
__________________________________________________________________________________________________
duration (Dense)                (None, 70)           17990       lambda_17[0][0]                  
Total params: 1,059,939
Trainable params: 1,059,939
Non-trainable params: 0
__________________________________________________________________________________________________
prediction (1, 116)
pattern [113, 108, 53, 58, 113, 113, 86, 57, 57, 62, 74, 108, 102, 97, 113, 86, 57, 57, 102, 86, 101, 113, 85, 1

prediction (1, 116)
pattern [58, 113, 113, 86, 57, 57, 62, 74, 108, 102, 97, 113, 86, 57, 57, 102, 86, 101, 113, 85, 113, 53, 113, 80, 113, 103, 113, 97, 113, 92, 113, 62]
pattern2 [3, 5, 3, 3, 5, 3, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 6, 3, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1]
Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   
______

...

input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   
__________________________________________________________________________________________________
embedding_37 (Embedding)        (None, None, 100)    7000        input_38[0][0]                   
__________________________________________________________________________________________________
concatenate_18 (Concatenate)    (None, None, 200)    0           embedding_36[0][0]               
                                                                 embedding_37[0][0]               
__________

concatenate_18 (Concatenate)    (None, None, 200)    0           embedding_36[0][0]               
                                                                 embedding_37[0][0]               
__________________________________________________________________________________________________
lstm_36 (LSTM)                  (None, None, 256)    467968      concatenate_18[0][0]             
__________________________________________________________________________________________________
lstm_37 (LSTM)                  (None, None, 256)    525312      lstm_36[0][0]                    
__________________________________________________________________________________________________
dense_19 (Dense)                (None, None, 1)      257         lstm_37[0][0]                    
__________________________________________________________________________________________________
reshape_18 (Reshape)            (None, None)         0           dense_19[0][0]                   
__________

prediction (1, 116)
pattern [102, 113, 74, 113, 74, 113, 102, 113, 96, 113, 74, 113, 63, 113, 63, 113, 97, 113, 53, 113, 108, 113, 59, 113, 81, 113, 113, 113, 112, 113, 107, 113]
pattern2 [3, 1, 4, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 3, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 4, 1]
Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   

prediction (1, 116)
pattern [74, 113, 102, 113, 96, 113, 74, 113, 63, 113, 63, 113, 97, 113, 53, 113, 108, 113, 59, 113, 81, 113, 113, 113, 112, 113, 107, 113, 91, 113, 63, 113]
pattern2 [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 3, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 4, 1, 4, 1, 4, 1, 2, 1]
Model: "functional_69"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, None, 100)    11600       input_37[0][0]                   


done
